In [2]:
import pandas as pd 

In [2]:
df2022=pd.read_csv('2022.1.1-2022.12.31',index_col=0)
df2021=pd.read_csv('2021.1.1-2021.12.31',index_col=0)
df2020=pd.read_csv('2020.1.1-2020.12.31',index_col=0)
df2019=pd.read_csv('2019.1.1-2019.12.31',index_col=0)
df2018=pd.read_csv('2018.1.1-2018.12.31',index_col=0)
df2017=pd.read_csv('2017.1.1-2017.12.31',index_col=0)
df2016=pd.read_csv('2016.1.1-2016.12.31',index_col=0)
df2015=pd.read_csv('2015.1.1-2015.12.31',index_col=0)
df2014=pd.read_csv('2014.1.1-2014.12.31',index_col=0)
planillas=pd.concat([df2022,df2021,df2020,df2019,df2018,df2017,df2016,df2015,df2014])


In [3]:
planillas['Fecha']=pd.to_datetime(planillas['Fecha'])

In [5]:
planillas.nunique()

Fecha               2407
Inicio               763
hora_Resultado       769
Fin                  771
Tipo                   4
Uso                    4
Categoría             10
Certificado       146817
Vence               3638
Dominio            82247
Resultado              4
CUIT               60843
Razón Social       61337
Línea                  2
dtype: int64

In [6]:
planillas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176857 entries, 1153444 to 1004193
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Fecha           176857 non-null  datetime64[ns]
 1   Inicio          176857 non-null  object        
 2   hora_Resultado  176857 non-null  object        
 3   Fin             176857 non-null  object        
 4   Tipo            176857 non-null  object        
 5   Uso             176857 non-null  object        
 6   Categoría       175232 non-null  object        
 7   Certificado     146820 non-null  object        
 8   Vence           176314 non-null  object        
 9   Dominio         176857 non-null  object        
 10  Resultado       176857 non-null  object        
 11  CUIT            175162 non-null  float64       
 12  Razón Social    175144 non-null  object        
 13  Línea           176857 non-null  int64         
dtypes: datetime64[ns](1), float64

Escribir a csv 

In [8]:
planillas.to_csv('Lista_planillas_COMPLETE.csv')

# Importar librerias

In [7]:
import time 

In [8]:
import requests

In [9]:
from tqdm.notebook import tqdm

In [10]:
from bs4 import BeautifulSoup

# Definir funciones

Request Historico de Revisiones 

In [11]:
def get_detalles(Patente):#input is in string format 
    global soup
    url = "https://rto.cent.gov.ar/rto/RTO/buscarHistorico"

    payload = "dominio="+Patente+"&limite=5&codigoPais=AR&="
    headers = {
        "cookie": "JSESSIONID=B544913AE18097DC002DA2F72AC68FA6",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:108.0) Gecko/20100101 Firefox/108.0",
        "Accept": "text/javascript, text/html, application/xml, text/xml, */*",
        "Accept-Language": "es-AR,es;q=0.8,en-US;q=0.5,en;q=0.3",
        "Accept-Encoding": "gzip, deflate, br",
        "X-Requested-With": "XMLHttpRequest",
        "X-Prototype-Version": "1.6.1",
        "Content-type": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://rto.cent.gov.ar",
        "Connection": "keep-alive",
        "Referer": "https://rto.cent.gov.ar/rto/",
        "Cookie": cookie,
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "TE": "trailers"
    }

    response = requests.request("POST", url, data=payload, headers=headers)
    #print response text (test)
    #print(response.text)
    soup=BeautifulSoup(response.text,'lxml')

Limpiar soup para historico de revisiones

In [12]:

def clean_data(tags_hist_revisiones):# returns a list of values: tags
        tags_datos=soup.select(tags_hist_revisiones) #data is inside the tag td
        global datos
        datos=[] 
        for data in tags_datos:
            datos.append(data.get_text(strip=True)) #make a list with the values 
        return(datos)

Crear lista de diccionarios de historico de revisiones:cada diccionario es una fila 

In [13]:
def create_rows_hist(Patente):
    global df
    global rows
    num_row=int(len(datos)/7)#cantidad de filas(row). cada fila tiene 7 elementos
    for i in range(0,num_row):
        i*=7
        dic={'Dominio':Patente,
            'Planilla':datos[1+i].replace('.',''),
            'Taller':int(datos[0+i]),
            'Tipo Uso':datos[2+i],
            'Fecha':datos[3+i],
            'Resultado':datos[4+i],
            'Versión RTO':datos[5+i]
            }
        rows.append(dic)

Crear dataframe (df) de historico de revisiones 

In [14]:
def create_df_hist():# returns a dataframe df 
    global df 
    df=pd.DataFrame(rows)
    return(df)

Utilizando las funciones anteriores. las unimos en una unica funcion. Iteramos en la lista de Patentes

In [15]:
def scrape(Patentes):
    global rows
    rows=[]
    tags_hist_revisiones='td'#tag de historico de revisiones
    for Patente in tqdm(Patentes):
        get_detalles(str(Patente))
        clean_data(tags_hist_revisiones)
        create_rows_hist(Patente)
    create_df_hist()# returns dataframe df
    return(df['Planilla'].nunique())

Guardar los dataframes en archivos .csv

In [16]:
def to_csv(num):
    filename=str(num)+'planillas.hist'
    df.to_csv(filename)

Dividimos la lista de patentes en N listas. En caso de error no perdemos todo 


In [17]:
def div_lista(Patentes,part):
    global patentes_part
    n=int(len(Patentes)/part)
    patentes_part=[patentes[i:i+n]for i in range(0,len(patentes),n)]
    return(patentes_part)
    
    
    

Definimos una nueva funcion Scrape que crea un csv por cada particion de la lista de planillas. Esto nos ayuda a no tener que comenzar de 0 si el server se cae o se corta internet 

In [26]:
def scrape_part(Patentes,part,start):
    div_lista(Patentes,part)
    for i in tqdm(range(0+start,part)):
        scrape(patentes_part[i])
        to_csv(i)

        

Definimos una funcion para unir los archivos csv en un unico Dataframe y guardar como csv

In [28]:
def join_csv(part):
    global df_hist
    global planillas_hist
    planillas_hist=pd.DataFrame()
    for i in range(0,part):
        df_hist=pd.read_csv(str(i)+'planillas.hist',index_col=0)
        planillas_hist=pd.concat([planillas_hist,df_hist])
    planillas_hist.to_csv('0.0.planillas.hist.COMPLETE')


# Actualizar cookie

In [24]:
cookie="JSESSIONID=73DD0C000F144D328BDC83B6EDD489BA"

# Lista de PATENTES 

In [ ]:
patentes=planillas.sort_values(by=['Dominio'])['Dominio'].str.replace('*','',regex=False).unique().tolist()

# Scrape Particiones

In [27]:
scrape_part(patentes,30,7)

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/2561 [00:00<?, ?it/s]

  0%|          | 0/2561 [00:00<?, ?it/s]

  0%|          | 0/2561 [00:00<?, ?it/s]

  0%|          | 0/2561 [00:00<?, ?it/s]

...

  0%|          | 0/2561 [00:00<?, ?it/s]

  0%|          | 0/2561 [00:00<?, ?it/s]

  0%|          | 0/2561 [00:00<?, ?it/s]

  0%|          | 0/2561 [00:00<?, ?it/s]

  0%|          | 0/2561 [00:00<?, ?it/s]

# Verificar dataframe creado

In [ ]:
df_hist=pd.read_csv(str(29)+'planillas.hist',index_col=0)

En caso de no completar . Ejecutar de nuevo la funcion scrape_part pero modificandola para comenzar desde la ultima particion hecha correctamente 

# Unir las particiones en un unico archivo CSV

In [42]:
join_csv(30)


# Leer CSV completo y analizar 

In [3]:
planillas_hist=pd.read_csv('0.0.planillas.hist.COMPLETE',index_col=0)
planillas_hist.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 265738 entries, 0 to 11694
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Dominio      265738 non-null  object
 1   Planilla     265738 non-null  int64 
 2   Taller       265738 non-null  int64 
 3   Tipo Uso     265738 non-null  object
 4   Fecha        265738 non-null  object
 5   Resultado    265738 non-null  object
 6   Versión RTO  265738 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 16.2+ MB


In [44]:
planillas_hist.nunique()

Dominio         76341
Planilla       206842
Taller            127
Tipo Uso           10
Fecha            5132
Resultado           4
Versión RTO         2
dtype: int64